In [292]:
import numpy as np
import pandas as pd
import cv2
import mediapipe as mp
import face_recognition_models
import dlib
import os
from mediapipe.python.solutions.drawing_utils import _normalized_to_pixel_coordinates
from time import time
import matplotlib.pyplot as plt
import imutils
from imutils import face_utils
import face_recognition
import shutil

In [293]:
face_recognition_model = face_recognition_models.face_recognition_model_location()
face_encoder = dlib.face_recognition_model_v1(face_recognition_model)

In [294]:
landmark_points_68 = [162,234,93,58,172,136,149,148,152,377,378,365,397,288,323,454,389,71,63,105,66,107,336,
                  296,334,293,301,168,197,5,4,75,97,2,326,305,33,160,158,133,153,144,362,385,387,263,373,
                  380,61,39,37,0,267,269,291,405,314,17,84,181,78,82,13,312,308,317,14,87]
    
landmark_points_5_1 = [ 2, #bottom of nose tip
                     362, #left eye towards centre
                     263, #left eye away from centre
                     33,  #right eye away from centre
                     133 #right eye towards centre 
                    ]
landmark_points_5_2 = [ 2, #bottom of nose tip
                     263, #left eye away from centre
                     362, #left eye towards centre
                     133, #right eye towards centre 
                     33  #right eye away from centre
                    ]


In [295]:
mp_drawing = mp.solutions.drawing_utils

mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(model_selection=1,min_detection_confidence=0.7)

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, refine_landmarks=False,max_num_faces=1,min_detection_confidence=0.5)

### MP + Dlib utils


In [296]:
def landmark_pt_list(mesh_results,width,height):
    landmark_points_68 = [162,234,93,58,172,136,149,148,152,377,378,365,397,288,323,454,389,71,63,105,66,107,336,
                  296,334,293,301,168,197,5,4,75,97,2,326,305,33,160,158,133,153,144,362,385,387,263,373,
                  380,61,39,37,0,267,269,291,405,314,17,84,181,78,82,13,312,308,317,14,87]
    
    landmark_points_5_1 = [ 2, #bottom of nose tip
                     362, #left eye towards centre
                     263, #left eye away from centre
                     33,  #right eye away from centre
                     133 #right eye towards centre 
                    ]
    landmark_points_5_2 = [ 2, #bottom of nose tip
                     263, #left eye away from centre
                     362, #left eye towards centre
                     133, #right eye towards centre 
                     33  #right eye away from centre
                    ]
    landmark_points_5 = [ 263, #left eye away from centre
                       362, #left eye towards centre
                       33,  #right eye away from centre
                       133, #right eye towards centre
                        2 #bottom of nose tip 
                    ]

    if mesh_results.multi_face_landmarks:
#         print(mesh_results.multi_face_landmarks)
        for i,face_landmarks in enumerate(mesh_results.multi_face_landmarks): 
            if i==0:
                raw_landmark_set = []
                for index in landmark_points_5:
                    x = int(face_landmarks.landmark[index].x * width)
                    y = int(face_landmarks.landmark[index].y * height)
                    landmark_point=dlib.point([x,y])
                    raw_landmark_set.append(landmark_point)
                    #display(landmark_point)
                all_points=dlib.points(raw_landmark_set)
#         return dlib.points([{
#             "nose_tip": [raw_landmark_set[0]],
#             "left_eye": raw_landmark_set[1:3],
#             "right_eye": raw_landmark_set[3:],
#         }])
        return all_points

def bounding_rect(detection_results,width,height):
    if detection_results.detections:
        for i,detection in enumerate(detection_results.detections):
            if i==0:
                # bbox data
                bbox = detection.location_data.relative_bounding_box
                xy_min = _normalized_to_pixel_coordinates(bbox.xmin, bbox.ymin, width,height)
                xy_max = _normalized_to_pixel_coordinates(bbox.xmin + bbox.width, bbox.ymin + bbox.height,width,height)
                if xy_min is None or xy_max is None:
                    #print("face out of frame")
                    return
                else:
                    left,top =xy_min
                    right,bottom = xy_max
                    #bbox_points = { "xmin" : xmin,"ymin" : ymin,"xmax" : xmax,"ymax" : ymax}
                    rectangle= dlib.rectangle(left=left, bottom=bottom, right=right, top=top)
                    return rectangle

def ret_encoding(filepath,num_jitters=1):
    image_input = cv2.imread(filepath)
#     print("face test")
#     print(image_input.shape)
#     image_input =face_recognition.load_image_file(filepath)
#     image = cv2.imread(cap_path)        

    image_input = cv2.cvtColor(image_input, cv2.COLOR_BGR2RGB)
#     print(image_input.shape)

#     width,height=image_input.shape[:-1]
    height, width, _ = image_input.shape
    detection_results = face_detection.process(image_input)
    mesh_results = face_mesh.process(image_input)
    
    all_points=  landmark_pt_list(mesh_results,width,height)  
#     print("all_points", all_points)

    b_box=bounding_rect(detection_results,width,height)
#     print(b_box)
#     print(b_box.left(),b_box.right(),b_box.top(),b_box.bottom())

    if (all_points is None) or (b_box is None):
        return 
    raw_landmark_set=dlib.full_object_detection(b_box,all_points)
    #display(all_points)
    #display(b_box)
#     cv2.imshow("test", image_input)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    encodings=face_encoder.compute_face_descriptor(image_input, raw_landmark_set, num_jitters)

    return encodings



In [297]:
# ret_encoding(filepath="/Users/michaelmandiberg/Documents/projects-active/facemap_production/just_start_image/portrait-of-funny-afro-guy-picture-id1402424532.jpg")

### Linear sorting

In [298]:
def get_img_list(folder):
    img_list=[]
    for file in os.listdir(folder):
        if not file.startswith('.') and os.path.isfile(os.path.join(folder, file)):
            filepath = os.path.join(folder, file)
            filepath=filepath.replace('\\' , '/')
            img_list.append(file)
    return img_list        
    print("got image list")

In [299]:
def get_d(enc1, enc2):
    enc1=np.array(enc1)
    print("enc1")
    print(enc1[0])
    enc2=np.array(enc2)
    print("enc2")
    print(enc2[0])
    d=np.linalg.norm(enc1 - enc2, axis=0)
    print("d")
    print(d)
    return d


In [300]:
# not df
def encode_list(folder, img_list):
    enc_dict={}
    for img in img_list:
        img_path = os.path.join(folder,img)
        encoding=(ret_encoding(img_path))
        enc_dict[img]=encoding 
    return enc_dict

In [301]:
# not df
def get_closest(folder, start_img, img_list, enc_dict):
    img_list.remove(start_img)
#     enc1=(ret_encoding(os.path.join(folder,start_img)))
    enc1=enc_dict[start_img]
    dist=[]
    dist_dict={}
    for img in img_list:
#         file2=filebase+str(i)+".jpg".replace('\\' , '/')
#         test_img = os.path.join(folder,img)
#         enc2=(ret_encoding(test_img))
        enc2 = enc_dict[img]
        if (enc1 is not None) and (enc2 is not None):
#             print(file2)
            d = get_d(enc1, enc2)
            print ("d is", str(d))
            dist.append(d)
            dist_dict[d]=img
    dist.sort()
#     print(len(dist))
    return dist[0], dist_dict[dist[0]]

In [302]:
def save_sorted(folder, image, counter, dist):
    sorted_name = "linear_sort_"+str(counter)+"_"+str(round(dist, 2))+".jpg"
    sortfolder="sorted2"
    newfolder = os.path.join(folder,sortfolder)
    old_name=os.path.join(folder,image)
    new_name=os.path.join(newfolder,sorted_name)
    if not os.path.exists(newfolder):
        os.makedirs(newfolder)
    shutil.copy(old_name, new_name)
    print('saved, ',sorted_name)


### dataframe creation and sorting

In [303]:
def encode_list_df(folder, img_list):
#     enc_dict={}
    csv_name="face_encodings.csv"
    col1="file_name"
    col2="encoding"
    curr=0
    total = len(img_list)

    # encodings column list for splitting
    col_list=[]
    for i in range(128):
        col_list.append(col2+str(i))

    #initializing the dataframe
    image_data=pd.DataFrame(columns=[col1, col2])

    
    for img in img_list:
        if curr%10==0:print(curr,"/",total)
        curr+=1
        filepath = os.path.join(folder,img)        
        filepath=filepath.replace('\\' , '/')  ## cv2 accepts files with "/" instead of "\"
        encodings=ret_encoding(filepath)
#         print(encodings)
        if encodings is not None:              ## checking if a face is found
            data=pd.DataFrame({col1:img,col2:[np.array(encodings)]})
            image_data = pd.concat([image_data,data],ignore_index=True)  

    #splitting the encodings column
    output_data = pd.DataFrame(image_data[col2].to_list(), columns=col_list)
    #adding the filename column and then puting it first
    output_data[[col1]]=pd.DataFrame(image_data[col1].tolist(),index=image_data.index)
    clms = output_data.columns.tolist()
    clms = clms[-1:] + clms[:-1]
    output_data=output_data[clms]
    # saving without index
    output_data.to_csv(csv_name, index=False)
    df = pd.read_csv(csv_name)
    return output_data

In [304]:
def get_closest_df(folder, start_img, df_enc):
    if start_img == "median":
        enc1 = df_enc.median().to_list()
        print("in median")
#         print(enc1)
    elif start_img == "start_site_image_id":
        print("start_site_image_id (this is what we are comparing to)")
        print(start_site_image_id)
        enc1 = df_enc.loc[start_site_image_id].to_list()
        print(enc1)
        
    else:
#         enc1 = get 2-129 from df via stimg key
        print("start_img key is (this is what we are comparing to):")
        print(start_img)
        enc1 = df_enc.loc[start_img].to_list()
        df_enc=df_enc.drop(start_img)
#         print("in new img",len(df_enc.index))
#         print(enc1)
    
#     img_list.remove(start_img)
#     enc1=enc_dict[start_img]
    
    dist=[]
    dist_dict={}
    
#     print("df_enc")
#     print(df_enc)
    
    for index, row in df_enc.iterrows():
#         print(row['c1'], row['c2'])
#     for img in img_list:
        enc2 = row
        print("this is the enc2 row passing in", index)
        if (enc1 is not None) and (enc2 is not None):
            d = get_d(enc1, enc2)
            print ("d is", str(d), "for", index)
            dist.append(d)
            dist_dict[d]=index
    dist.sort()
#     print(len(dist))
    return dist[0], dist_dict[dist[0]], df_enc

In [305]:
# test if new and old make a face
def is_face(image):
    # For static images:
    # I think this list is not used
    IMAGE_FILES = []
    with mp_face_detection.FaceDetection(model_selection=1, 
                                        min_detection_confidence=0.6
                                        ) as face_detection:
        # image = cv2.imread(file)
        # Convert the BGR image to RGB and process it with MediaPipe Face Detection.
#         detection_results = face_detection.process(image)

        results = face_detection.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        # Draw face detections of each face.
        if not results.detections:
            is_face = False
        else:
            is_face = True
        # annotated_image = image.copy()
        # for detection in results.detections:
        #     is_face = True
        #     print('Nose tip:')
        #     print(mp_face_detection.get_key_point(
        #       detection, mp_face_detection.FaceKeyPoint.NOSE_TIP))
        #     mp_drawing.draw_detection(annotated_image, detection)
        # cv2.imwrite('/tmp/annotated_image' + str(idx) + '.png', annotated_image)

        return is_face


In [306]:
# test if new and old make a face
def test_pair(last_file, new_file):
    try:
        img = cv2.imread(new_file)
        height, width, layers = img.shape
        size = (width, height)
        print('loaded img 1')
        
        last_img = cv2.imread(new_file)
        last_height, last_width, last_layers = last_img.shape
        last_size = (last_width, last_height)
        print('loaded img 2')
        
        # test to see if this is actually an face, to get rid of blank ones/bad ones
        if is_face(img):
            print('new file is face')
            # if not the first image
#             if i>0:
            # blend this image with the last image
            blend = cv2.addWeighted(img, 0.5, last_img, 0.5, 0.0)
            print('blended faces')
            blended_face = is_face(blend)
            print('is_face ',blended_face)
            # if blended image has a detectable face, append the img
            if blended_face:
#                     img_array.append(img)
                print('is a face! adding it')
                return True
            else:
                print('skipping this one')
                return False
            # for the first one, just add the image
            # this may need to be refactored in case the first one is bad?
#             else:
#                 print('this is maybe the first round?')
#                 img_array.append(img)
        else:
            print('new_file is not face: ',new_file)
            return False

#         i+=1

    except:
        print('failed:',new_file)
        return False


In [307]:
folder="/Users/michaelmandiberg/Documents/projects-active/facemap_production/just_start_image"


img_list = get_img_list(folder)

# start_img = img_list[1]
start_img = "median"

start_img = "start_site_image_id"
# start_site_image_id = "faceimg_crop1_X-15toX5_Y-4toY4_Z-3toZ3_maxResize0.3_ct2412_1_1402424532.jpg"
start_site_image_id = "portrait-of-funny-afro-guy-picture-id1402424532.jpg"



# faceimg_crop1_X-15toX5_Y-4toY4_Z-3toZ3_maxResize0.3_ct2412_18_1396511988.jpg

# enc_dict = encode_list(folder, img_list)

df_enc = encode_list_df(folder, img_list)
print(df_enc)

df_enc.set_index('file_name', inplace=True)


#with lists/dicts
# dist=0
# for i in range(len(img_list)-1):
#     save_sorted(folder, start_img, i, dist)
#     dist, start_img = get_closest(folder, start_img,img_list, enc_dict)
    
#     print(dist)
#     print (start_img)
    
#     if dist > .37: 
#         continue

#with df
dist=0
print(len(df_enc.index))


    
for i in range(len(df_enc.index)-1):
    dist, start_img, df_enc = get_closest_df(folder, start_img,df_enc)
#     print(folder, start_img, i, dist)
    save_sorted(folder, start_img, i, dist)
#     print(i)
#     print(len(df_enc.index))
#     print(dist)
#     print (start_img)
    
    if dist > .60: 
        break
    
    
# I don't know why, but this isn't working
# for i in range(len(df_enc.index)-2):
#     dist, start_img, df_enc = get_closest_df(folder, start_img,df_enc)
# #     print(folder, start_img, i, dist)
#     save_sorted(folder, start_img, i, dist)

#     if i>0:
#         #test blend
# #         last_file = os.path.join(folder,)
#         blend_is_face = (test_pair(os.path.join(folder,last_img), os.path.join(folder,start_img)))
#         print('blend_is_face ',blend_is_face)
#         if blend_is_face:
# #         print(test_pair(last_img,start_img))
#             save_sorted(folder, start_img, i, dist)
#             last_img = start_img
# #         else:
# #             start_img = last_img
            
#     print(i)
#     print(len(df_enc.index))
#     print(dist)
#     print (start_img)
    
#     if dist > .37: 
#         break
        
       

    

0 / 4
                                           file_name  encoding0  encoding1  \
0  indian-boy-casual-cheerful-little-isolated-on-...  -0.197039   0.008390   
1  emotional-african-american-man-with-african-ha...  -0.157075   0.066053   
2  photo-of-cheerful-curly-girl-with-positive-emo...  -0.195631   0.116353   
3  portrait-of-funny-afro-guy-picture-id140242453...  -0.169882   0.125728   

   encoding2  encoding3  encoding4  encoding5  encoding6  encoding7  \
0   0.009492  -0.052163  -0.088729   0.013810  -0.065856  -0.103480   
1   0.035424  -0.077837  -0.051731   0.004132  -0.004017  -0.055078   
2   0.032778  -0.075347  -0.128841  -0.044508  -0.011787  -0.054210   
3   0.040373  -0.095802  -0.005333   0.004143   0.002504  -0.057213   

   encoding8  ...  encoding118  encoding119  encoding120  encoding121  \
0   0.170433  ...     0.065606     0.013003    -0.011903    -0.095272   
1   0.167490  ...     0.099749     0.098921     0.000436     0.016024   
2   0.305887  ...     0.0426